# Exploratory Data Analysis
This dataset was scraped from Airbnb using Octoparse through a 2-step method, treated, merged and processed. You can check this all in the `preprocessing.ipynb` notebook in the repository. 

Our objective is to use it to analyse and understand the Airbnb market in Campos do Jordão, a touristic town in the state of São Paulo, Brazil.

In [35]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [42]:
# Loading the dataset
df = pd.read_csv('datasets/final_dataset.csv')
df = df.drop(df.columns[0], axis=1)  # dropping first column (old index)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 643 entries, 0 to 642
Data columns (total 46 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   roomType                           643 non-null    object 
 1   roomPrice                          643 non-null    float64
 2   roomURL                            643 non-null    object 
 3   hostType                           643 non-null    object 
 4   rating                             643 non-null    float64
 5   countReviews                       643 non-null    float64
 6   Air Conditioning                   643 non-null    int64  
 7   TV                                 643 non-null    int64  
 8   Dryer                              643 non-null    int64  
 9   Bathroom                           643 non-null    int64  
 10  Ethernet connection                643 non-null    int64  
 11  Kitchen                            643 non-null    int64  

## 